In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import joblib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [9]:
def preprocess(data):
    
    scaler = joblib.load('minMaxScaler')
    som_median = data['modify_soms.'].median()
    twist_median = data['modify_twist'].median()
    predScore_median = data['predscore'].median()
    
    data['pred_somersault'] = np.where(data['modify_soms.'].notnull(),data['modify_soms.'],
                                      np.where((data['fine_grained_soms.'].isnull()),0,som_median))

    data['pred_twist'] = np.where(data['modify_twist'].notnull(),data['modify_twist'],
                                      np.where((data['fine_grained_twist'].isnull()),0,twist_median))

    data['updated_predscore'] = data['predscore'].fillna(predScore_median)
    columnsToTrain = ['pred_twist','bend_angle','entry_angle','pred_somersault','updated_predscore',
                 'mean_splash_index','ctr_area']
    data_df = data[columnsToTrain]
    data_df = scaler.transform(data_df)
    print('Preprocessing Complete')
    return data_df

In [16]:
def final_results(file):
    data = pd.read_excel(file)
    preprocessed_data = preprocess(data)
    model = joblib.load('svr')
    svr_pred = round(model.predict(preprocessed_data)[0],4)
    print('Modelling Complete')
    return svr_pred

In [17]:
svr_pred = final_results('Test_SVR.xlsx')
print(svr_pred)

Preprocessing Complete
Modelling Complete
0.7475
